In [1]:
import geopandas as gpd
import json

cutblock_df = gpd.read_file("data/alberta_cutblocks.shp")
ab_df = gpd.read_file("data/alberta_boundary.shp")

In [2]:
fmu_name = "S10"
fmu = cutblock_df.query(f"FMU_NAME=='{fmu_name}'")
fmu = fmu.dissolve(by='FMU_NAME')
fmu.centroid

FMU_NAME
S10    POINT (479085.290 6251500.838)
dtype: geometry

In [3]:
from ipyleaflet import Map, GeoData, basemaps, ScaleControl, projections
from ipywidgets import Layout

centroid = fmu.centroid.to_crs(epsg=4326) # Get the centroid of the fmu in decimal degrees
center = centroid[0] # get the first point
center = (center.y, center.x) # Convert it lat, lon

# center
m = Map(center=center, zoom=9.5, 
        basemap=basemaps.Esri.WorldImagery, 
        layout=Layout(width="100%", height="600px"))

cutblocks = GeoData(geo_dataframe=fmu.to_crs(epsg=4326),
                    style={'color': '#00FF00', 'fillOpacity': 0, 'weight': 3},
                    name=f'Cutblocks (FMU = {fmu_name})')

m.add_layer(cutblocks)
m.add_control(ScaleControl(position='bottomleft'))
m

Map(center=[56.430113818355224, -115.33925880126449], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [4]:
import ee
ee.Initialize()
gfc = ee.Image("UMD/hansen/global_forest_change_2019_v1_7")
gfc

In [5]:
vis_params = {
    "bands": ["lossyear"],
    "min": 0,
    "max": 19,
    "palette": ["yellow", "red"]
}

url = gfc.getMapId(vis_params)["tile_fetcher"].url_format

In [6]:
from ipyleaflet import TileLayer
tile_layer = TileLayer(url=url)
m.add_layer(tile_layer)
m

Map(center=[56.430113818355224, -115.33925880126449], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [7]:
for cutblock in cutblock_df:
    print(cutblock)

FMU_NAME
Shape_Leng
Shape_Area
geometry
